<a href="https://colab.research.google.com/github/BossDevinator/IS_4482/blob/main/A7_Moore_Devan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ASSIGNMENT 7
##Devan Moore
###10/7/2024

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [32]:
!cp "/content/drive/MyDrive/Colab Notebooks/A7_Moore_Devan.ipynb" ./
!jupyter nbconvert --to html "A7_Moore_Devan.ipynb"

[NbConvertApp] Converting notebook A7_Moore_Devan.ipynb to html
[NbConvertApp] Writing 703034 bytes to A7_Moore_Devan.html


In [1]:
import pandas as pd
from sklearn import tree, metrics
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix,\
 recall_score, precision_score, f1_score, accuracy_score, make_scorer,\
  precision_recall_fscore_support
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import numpy as np
from decimal import getcontext, Decimal
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
import random

In [2]:
import warnings
warnings.filterwarnings('ignore')

pd.options.display.float_format = '{:.3f}'.format

random_state = 13

##TASK 1:
###Prepare Data

###Data Import

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/matthewpecsok/4482_fall_2024/main/data/CD_additional_modified.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4117 entries, 0 to 4116
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             4117 non-null   int64  
 1   job             4117 non-null   object 
 2   marital         4117 non-null   object 
 3   education       4117 non-null   object 
 4   default         4117 non-null   object 
 5   housing         4117 non-null   object 
 6   loan            4117 non-null   object 
 7   contact         4117 non-null   object 
 8   month           4117 non-null   object 
 9   day_of_week     4117 non-null   object 
 10  duration        4117 non-null   int64  
 11  campaign        4117 non-null   int64  
 12  pdays           4117 non-null   int64  
 13  previous        4117 non-null   int64  
 14  poutcome        4117 non-null   object 
 15  emp_var_rate    4117 non-null   float64
 16  cons_price_idx  4117 non-null   float64
 17  cons_conf_idx   4117 non-null   f

In [5]:
df.describe()

,age,duration,campaign,pdays,previous,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed
count,4117.000,4117.000,4117.000,4117.000,4117.000,4117.000,4117.000,4117.000,4117.000,4117.000
mean,40.115,256.850,2.537,960.403,0.190,0.086,93.580,-40.501,3.622,5166.497
std,10.315,254.750,2.569,191.968,0.542,1.563,0.579,4.593,1.733,73.671
min,18.000,0.000,1.000,0.000,0.000,-3.400,92.201,-50.800,0.635,4963.600
25%,32.000,103.000,1.000,999.000,0.000,-1.800,93.075,-42.700,1.334,5099.100
50%,38.000,181.000,2.000,999.000,0.000,1.100,93.749,-41.800,4.857,5191.000
75%,47.000,317.000,3.000,999.000,0.000,1.400,93.994,-36.400,4.961,5228.100
max,88.000,3643.000,35.000,999.000,6.000,1.400,94.767,-26.900,5.045,5228.100


In [6]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,30,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,...,2,999,0,nonexistent,-1.800,92.893,-46.200,1.313,5099.100,no
1,39,services,single,high.school,no,no,no,telephone,may,fri,...,4,999,0,nonexistent,1.100,93.994,-36.400,4.855,5191.000,no
2,25,services,married,high.school,no,yes,no,telephone,jun,wed,...,1,999,0,nonexistent,1.400,94.465,-41.800,4.962,5228.100,no
3,38,services,married,basic.9y,no,unknown,unknown,telephone,jun,fri,...,3,999,0,nonexistent,1.400,94.465,-41.800,4.959,5228.100,no
4,47,admin.,married,university.degree,no,yes,no,cellular,nov,mon,...,1,999,0,nonexistent,-0.100,93.200,-42.000,4.191,5195.800,no


###Data Prep

In [7]:
y_target = df.pop('y')

In [8]:
y_target = y_target.eq('yes').mul(1)
y_target.head(7)

,y
0,0
1,0
2,0
3,0
4,0
5,0
6,0


In [9]:
df.dtypes

,0
age,int64
job,object
marital,object
education,object
default,object
housing,object
loan,object
contact,object
month,object
day_of_week,object


There are no numerical columns that need to be converted to categorical before one-hot encoding.

In [10]:
df_encoded = pd.get_dummies(df)
df_encoded.head()

,age,duration,campaign,pdays,previous,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,...,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,30,487,2,999,0,-1.800,92.893,-46.200,1.313,5099.100,...,False,False,True,False,False,False,False,False,True,False
1,39,346,4,999,0,1.100,93.994,-36.400,4.855,5191.000,...,False,False,True,False,False,False,False,False,True,False
2,25,227,1,999,0,1.400,94.465,-41.800,4.962,5228.100,...,False,False,False,False,False,False,True,False,True,False
3,38,17,3,999,0,1.400,94.465,-41.800,4.959,5228.100,...,False,False,True,False,False,False,False,False,True,False
4,47,58,1,999,0,-0.100,93.200,-42.000,4.191,5195.800,...,False,False,False,True,False,False,False,False,True,False


##TASK 2:
### ANN Models

In [11]:
tree_model_ccp_0 = tree.DecisionTreeClassifier(ccp_alpha=0)
tree_model_ccp_5 = tree.DecisionTreeClassifier(ccp_alpha=0.5)
tree_model_ccp_10 = tree.DecisionTreeClassifier(ccp_alpha=1)

In [12]:
model_list = [
    tree_model_ccp_0,
    tree_model_ccp_5,
    tree_model_ccp_10
]

In [13]:
for model in model_list:
  scores = cross_validate(
    model, df_encoded, y_target, cv=3, scoring=['accuracy','precision','recall','f1'],return_train_score=True)
  df = pd.DataFrame(scores)
  df = df[['test_accuracy','train_accuracy']]
  df['train-test'] = df.train_accuracy-df.test_accuracy
  print(f'for ccp_alpha={model.ccp_alpha}\n {df.mean()}\n')

for ccp_alpha=0
 test_accuracy    0.890
train_accuracy   1.000
train-test       0.110
dtype: float64

for ccp_alpha=0.5
 test_accuracy     0.890
train_accuracy    0.890
train-test       -0.000
dtype: float64

for ccp_alpha=1
 test_accuracy     0.890
train_accuracy    0.890
train-test       -0.000
dtype: float64



In [14]:
mlp_model_1 = MLPClassifier(hidden_layer_sizes=(5,), random_state=random_state)
mlp_model_2 = MLPClassifier(hidden_layer_sizes=(25,25), random_state=random_state)
mlp_model_3 = MLPClassifier(hidden_layer_sizes=(20,7,7), random_state=random_state)
mlp_model_4 = MLPClassifier(hidden_layer_sizes=(15,15), random_state=random_state)
mlp_model_5 = MLPClassifier(hidden_layer_sizes=(5,5), random_state=random_state)

In [15]:
mlp_models = [
    mlp_model_1,
    mlp_model_2,
    mlp_model_3,
    mlp_model_4,
    mlp_model_5
]

In [16]:
for model in mlp_models:
  scores = cross_validate(
      model, df_encoded, y_target, cv=3, scoring=['accuracy','precision','recall','f1'],return_train_score=True
  )
  df_scores = pd.DataFrame(scores)
  df_scores = df_scores[['test_accuracy', 'train_accuracy']]
  df['train-test'] = df_scores.train_accuracy - df_scores.test_accuracy
  print(f'MLP Model Hidden Layers: {model.hidden_layer_sizes})\n{df_scores.mean()}\n')

MLP Model Hidden Layers: (5,))
test_accuracy    0.892
train_accuracy   0.893
dtype: float64

MLP Model Hidden Layers: (25, 25))
test_accuracy    0.849
train_accuracy   0.847
dtype: float64

MLP Model Hidden Layers: (20, 7, 7))
test_accuracy    0.905
train_accuracy   0.906
dtype: float64

MLP Model Hidden Layers: (15, 15))
test_accuracy    0.900
train_accuracy   0.903
dtype: float64

MLP Model Hidden Layers: (5, 5))
test_accuracy    0.908
train_accuracy   0.907
dtype: float64



After evaluating each of the models above, the one MLP model to hold the highest average accuracy with the smallest variation between train and text accuracy is the model with 2 hidden layers and 5 neurons each. They hold to a 0.908 test_accuracy and a 0.907 train_accuracy.

The least effective model was the one hosing 2 hidden layers with 25 nuerons each.

##TASK 3:

In [17]:
mlp_model_new_1 = MLPClassifier(hidden_layer_sizes=(5,5), learning_rate='adaptive', activation='tanh', solver='adam', random_state=random_state)
mlp_model_new_2 = MLPClassifier(hidden_layer_sizes=(5,5), learning_rate='invscaling', activation='logistic', solver='sgd', random_state=random_state)

In [18]:
new_mlp_models = [
    mlp_model_new_1,
    mlp_model_new_2
]

In [19]:
for model in new_mlp_models:
  scores = cross_validate(
      model, df_encoded, y_target, cv=3, scoring=['accuracy','precision','recall','f1'],return_train_score=True
  )
  df_scores = pd.DataFrame(scores)
  df_scores = df_scores[['test_accuracy', 'train_accuracy']]
  df['train-test'] = df_scores.train_accuracy - df_scores.test_accuracy
  print(f'MLP Model Activation: {model.activation}, Solver: {model.solver}, Learning_Rate: {model.learning_rate})\n{df_scores.mean()}\n')

MLP Model Activation: tanh, Solver: adam, Learning_Rate: adaptive)
test_accuracy    0.890
train_accuracy   0.890
dtype: float64

MLP Model Activation: logistic, Solver: sgd, Learning_Rate: invscaling)
test_accuracy    0.890
train_accuracy   0.890
dtype: float64



We can see that the accuracy did not improve from the previous model despite changing the activation, solver, and learning rate on each model.

##TASK 4:

In [20]:
parameters = {
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['sgd', 'adam','lbfgs'],
    'learning_rate': ['constant', 'invscaling', 'adaptive']
}

In [21]:
mlp = MLPClassifier(random_state=random_state)
grid_search = GridSearchCV(mlp, parameters, cv=3, scoring='accuracy', return_train_score=True)
grid_search.fit(df_encoded, y_target)

GridSearchCV(cv=3, estimator=MLPClassifier(random_state=13),
             param_grid={'activation': ['logistic', 'tanh', 'relu'],
                         'learning_rate': ['constant', 'invscaling',
                                           'adaptive'],
                         'solver': ['sgd', 'adam', 'lbfgs']},
             return_train_score=True, scoring='accuracy')

In [22]:
grid_search_results = pd.DataFrame(grid_search.cv_results_)
grid_search_results = grid_search_results[['rank_test_score','mean_test_score','mean_train_score']]
grid_search_results['mean_score_diff_test_train'] = grid_search_results.mean_test_score - grid_search_results.mean_train_score
grid_search_results = grid_search_results.sort_values(by='rank_test_score', ascending=True)
grid_search_results.head()

,rank_test_score,mean_test_score,mean_train_score,mean_score_diff_test_train
1,1,0.906,0.907,-0.001
4,1,0.906,0.907,-0.001
7,1,0.906,0.907,-0.001
2,4,0.905,0.906,-0.001
5,4,0.905,0.906,-0.001


##TASK 5:

In [28]:
svc_parameters = {
    'C': [0.1, 1, 10, 100, 1000],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
}

In [29]:
svc = SVC(random_state=random_state)
svc_model = GridSearchCV(svc, svc_parameters,scoring='accuracy', cv=3, return_train_score=True)
svc_model.fit(df_encoded, y_target)

GridSearchCV(cv=3, estimator=SVC(random_state=13),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             return_train_score=True, scoring='accuracy')

In [30]:
svc_results = pd.DataFrame(svc_model.cv_results_)
svc_results = svc_results[['rank_test_score','mean_test_score','mean_train_score']]
svc_results['mean_score_diff_test_train'] = svc_results.mean_test_score - svc_results.mean_train_score
svc_results = svc_results.sort_values(by='rank_test_score', ascending=True)
svc_results.head()

,rank_test_score,mean_test_score,mean_train_score,mean_score_diff_test_train
8,1,0.911,0.914,-0.002
4,2,0.911,0.916,-0.005
0,3,0.911,0.916,-0.005
12,4,0.910,0.912,-0.002
16,5,0.909,0.913,-0.005


##TASK 6:

Overall, this assignment was really interesting. It was cool to work through the various levels of models, helping to find that the most successful model was a 3-fold cross validation with 2 layers, 5 neurons each. Then being able to build on top of that to finally see that when if came down to the SVC test number 8 was the most accurate with the highest mean_test_score.